In [2]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/new_data/final.csv')

In [4]:
punct_to_remove = string.punctuation
stop_words = set(stopwords.words('english'))

In [5]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','', punct_to_remove))
def remove_stopwords(text):
    return ' '.join([word for word in str(text).split() if word not in stop_words])

In [6]:
df['description'] = df['description'].apply(lambda text: remove_stopwords(text))

In [7]:
df['title'] = df['title'].apply(lambda text: remove_punctuation(text))
df['director'] = df['director'].apply(lambda text: remove_punctuation(text))
df['description'] = df['description'].apply(lambda text: remove_punctuation(text))

In [8]:
df['title'] =df['title'].astype(str)
df['title'] =df['title'].str.lower()

df['description'] = df['description'].astype(str)
df['description'] = df['description'].str.lower()

df['director'] = df['director'].astype(str)
df['director'] = df['director'].str.lower()

In [9]:
df['text'] = df['title'] + ' ' + df['director'] + ' ' + df['description']

In [10]:
train_data, test_data = train_test_split(df, test_size =0.2, random_state=1)

In [11]:
vectorizer = CountVectorizer(ngram_range=(1, 5))
counts = vectorizer.fit_transform(train_data['text'].values)
classifier = MultinomialNB()
targets = train_data['profitable'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
test = test_data['text']
example_counts = vectorizer.transform(test)
predictions = classifier.predict(example_counts)

In [13]:
df_nb = test_data.copy()
df_nb['nb_prediction'] = predictions

In [14]:
df_nb['correct'] = np.where(df_nb['nb_prediction'] == df_nb['profitable'], 1, 0)

In [14]:
df_nb.to_csv('../data/new_data/nb.csv')

In [31]:
true_positives = np.where((df_nb['nb_prediction'] == 1) & (df_nb['profitable'] == 1), 1,0)
true_negatives = np.where((df_nb['nb_prediction'] == 0) & (df_nb['profitable'] == 0), 1,0)
false_positives = np.where((df_nb['nb_prediction'] == 1) & (df_nb['profitable'] == 0), 1,0)
false_negatives = np.where((df_nb['nb_prediction'] == 0) & (df_nb['profitable'] == 1), 1,0)

In [36]:
accuracy = (true_positives.sum()+true_negatives.sum())/df_nb.title.count()
precision = true_positives.sum()/(true_positives.sum()+false_positives.sum())
recall = true_positives.sum()/(true_positives.sum()+false_negatives.sum())

In [30]:
precision

0.6290956749672346

In [33]:
print(true_positives.sum())
print(true_negatives.sum())
print(false_positives.sum())
print(false_negatives.sum())

480
65
283
62


In [38]:
print(accuracy)
print(precision)
print(recall)

0.6123595505617978
0.6290956749672346
0.8856088560885609
